In [1]:
file = open('d://bitmex_data_1m.csv','r')
print(file.readline())
for i in range(100):
    file.readline()

file.readline()

timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional



'2015-09-25 13:41:00+00:00,XBTUSD,237.45,237.45,237.45,237.45,0,0,,,0,0.0,0.0\n'

In [2]:
import pandas as pd
df = pd.read_csv('d:/bitmex_data_1m.csv', delimiter=',')

In [3]:
import datetime 
import time

#df의 timestamp 행을 Windowing 하기위해 int 형태로 바꾸는 함수
def convert_to_timestamp(row):
    return time.mktime(datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S').timetuple())

#+00:00 를 없애고 KST 로 바꾸는 함수
def convert_gmt_to_kst(row):
    return str(datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S+00:00')+datetime.timedelta(hours=9)-datetime.timedelta(minutes=1))

#timestamp int 형태를 string 형태의 yyy-MM-dd HH:mm:ss 로 바꾸기
def convert_timestamp_to_datetime_str(row):
    return str(datetime.datetime.fromtimestamp(row))
#time.mktime(datetime.datetime.strptime(df['timestamp'][0], '%Y-%m-%d %H:%M:%S+00:00').timetuple())

In [4]:
df

,timestamp,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
0,2015-09-25 12:01:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
1,2015-09-25 12:02:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2,2015-09-25 12:03:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
3,2015-09-25 12:04:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
4,2015-09-25 12:05:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005552,2023-05-08 03:13:00+00:00,XBTUSD,28320.5,28328.5,28320.5,28328.5,12,50000,28322.75,200.0,176536630,1.765366,50000.0
4005553,2023-05-08 03:14:00+00:00,XBTUSD,28328.5,28330.0,28329.5,28330.0,11,43500,28329.65,200.0,153549633,1.535496,43500.0
4005554,2023-05-08 03:15:00+00:00,XBTUSD,28330.0,28329.5,28320.0,28320.0,51,173600,28326.10,100.0,612862282,6.128623,173600.0
4005555,2023-05-08 03:16:00+00:00,XBTUSD,28320.0,28322.5,28289.5,28304.0,154,1151700,28313.05,3300.0,4067744000,40.677440,1151700.0


In [7]:
def get_processed_df_from_raw(df,base_delta, from_date=None):
    if(from_date):
        df = df.loc[df.timestamp >= from_date]  #일단 GMT 니까 1분 뒤로 조정할 걸 생각하고 +1분부터 가져오면 된다.
    
    #kst 로 바꾸고 기존 gmt 형태는 drop
    df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)
    
    df = df[['timestamp_kst','low','high','close']]
    
    df = df.reset_index()
    
    #int 형태의 timestamp 열도 추가
    df['timestamp_int'] = df['timestamp_kst'].apply(convert_to_timestamp)
    
    #다시 필요한 컬럼만 정돈
    df = df[['timestamp_kst','timestamp_int','low','high','close']]
    
    #최종 x분봉의 형태구현 base_delta = 15, 45, 240, 1440
    final_df = df[['low']].rolling(window=base_delta).min()     
    final_df['high'] = df[['high']].rolling(window=base_delta).max() 
    final_df['close'] = df['close']                         
    final_df['timestamp_int'] = df[['timestamp_int']]-(base_delta*60-60)  
    
    final_df = final_df[base_delta-1:] #window수 -1 값만큼 버리고
    final_df['timestamp_kst'] = final_df['timestamp_int'].apply(convert_timestamp_to_datetime_str)
    final_df = final_df.reset_index()[['timestamp_kst','low','high','close','timestamp_int']] #리셋재구성
    
    return final_df[final_df['timestamp_int']%(base_delta*60)==0]
    

In [8]:
#15분봉
get_processed_df_from_raw(df,15,'2022-12-31 15:01:00')

C:\Users\EUISEOP\AppData\Local\Temp\ipykernel_13216\3200420179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)


,timestamp_kst,low,high,close,timestamp_int
0,2023-01-01 00:00:00,16583.5,16608.0,16600.0,1.672499e+09
15,2023-01-01 00:15:00,16583.5,16607.0,16590.5,1.672500e+09
30,2023-01-01 00:30:00,16582.5,16591.0,16586.0,1.672501e+09
45,2023-01-01 00:45:00,16578.0,16586.5,16580.0,1.672502e+09
60,2023-01-01 01:00:00,16579.5,16583.0,16583.0,1.672502e+09
...,...,...,...,...,...
183540,2023-05-08 11:00:00,28207.0,28374.0,28362.0,1.683511e+09
183555,2023-05-08 11:15:00,28293.0,28374.0,28329.5,1.683512e+09
183570,2023-05-08 11:30:00,28260.0,28330.5,28330.5,1.683513e+09
183585,2023-05-08 11:45:00,28315.5,28395.0,28376.0,1.683514e+09


In [9]:
#45분봉
get_processed_df_from_raw(df,45,'2022-12-31 15:01:00')

C:\Users\EUISEOP\AppData\Local\Temp\ipykernel_13216\3200420179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)


,timestamp_kst,low,high,close,timestamp_int
0,2023-01-01 00:00:00,16582.5,16608.0,16586.0,1.672499e+09
45,2023-01-01 00:45:00,16573.5,16588.5,16588.5,1.672502e+09
90,2023-01-01 01:30:00,16578.0,16596.5,16578.0,1.672504e+09
135,2023-01-01 02:15:00,16563.0,16580.0,16568.5,1.672507e+09
180,2023-01-01 03:00:00,16563.5,16568.5,16563.5,1.672510e+09
...,...,...,...,...,...
183375,2023-05-08 08:15:00,28465.5,28860.0,28497.0,1.683501e+09
183420,2023-05-08 09:00:00,28424.0,28690.0,28660.0,1.683504e+09
183465,2023-05-08 09:45:00,28509.5,28662.5,28528.0,1.683507e+09
183510,2023-05-08 10:30:00,28126.0,28652.5,28362.0,1.683509e+09


In [10]:
#4시간봉
get_processed_df_from_raw(df,240,'2022-12-31 15:01:00')

C:\Users\EUISEOP\AppData\Local\Temp\ipykernel_13216\3200420179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)


,timestamp_kst,low,high,close,timestamp_int
60,2023-01-01 01:00:00,16555.5,16596.5,16556.0,1.672502e+09
300,2023-01-01 05:00:00,16424.5,16555.5,16522.0,1.672517e+09
540,2023-01-01 09:00:00,16494.5,16540.0,16513.0,1.672531e+09
780,2023-01-01 13:00:00,16482.0,16528.5,16517.5,1.672546e+09
1020,2023-01-01 17:00:00,16489.5,16530.5,16530.5,1.672560e+09
...,...,...,...,...,...
182220,2023-05-07 13:00:00,28843.0,29005.5,28960.5,1.683432e+09
182460,2023-05-07 17:00:00,28758.0,29007.5,28905.5,1.683446e+09
182700,2023-05-07 21:00:00,28900.0,29141.0,28987.5,1.683461e+09
182940,2023-05-08 01:00:00,28840.5,29074.5,28989.0,1.683475e+09


In [11]:
#일봉
get_processed_df_from_raw(df,1440,'2022-12-31 15:01:00')

C:\Users\EUISEOP\AppData\Local\Temp\ipykernel_13216\3200420179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)


,timestamp_kst,low,high,close,timestamp_int
540,2023-01-01 09:00:00,16482.0,16618.0,16610.0,1.672531e+09
1980,2023-01-02 09:00:00,16544.5,16789.5,16670.0,1.672618e+09
3420,2023-01-03 09:00:00,16596.0,16765.0,16656.0,1.672704e+09
4860,2023-01-04 09:00:00,16639.5,16978.0,16842.5,1.672790e+09
6300,2023-01-05 09:00:00,16750.5,16872.0,16818.0,1.672877e+09
...,...,...,...,...,...
176220,2023-05-03 09:00:00,28090.0,29278.5,29044.5,1.683072e+09
177660,2023-05-04 09:00:00,28670.5,29400.0,28844.0,1.683158e+09
179100,2023-05-05 09:00:00,28701.5,29710.0,29524.5,1.683245e+09
180540,2023-05-06 09:00:00,28363.5,29869.0,28906.5,1.683331e+09


In [14]:
#주봉은 월요일 기준으로 맞추는게 일반적인데 최근 데이터의 일자로 맞추어져서 기준일이 안맞음.. 별도로 처리 필요
get_processed_df_from_raw(df,10080,'2022-12-31 15:01:00')

C:\Users\EUISEOP\AppData\Local\Temp\ipykernel_13216\3200420179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp_kst'] = df['timestamp'].apply(convert_gmt_to_kst)


,timestamp_kst,low,high,close,timestamp_int
6300,2023-01-05 09:00:00,16665.5,18000.0,17943.0,1.672877e+09
16380,2023-01-12 09:00:00,17909.5,21652.5,20671.0,1.673482e+09
26460,2023-01-19 09:00:00,20650.5,23903.0,23055.0,1.674086e+09
36540,2023-01-26 09:00:00,22368.5,23970.5,23734.0,1.674691e+09
46620,2023-02-02 09:00:00,22624.0,24264.0,22954.5,1.675296e+09
56700,2023-02-09 09:00:00,21351.0,24393.0,24352.0,1.675901e+09
66780,2023-02-16 09:00:00,23322.5,25300.0,24178.0,1.676506e+09
76860,2023-02-23 09:00:00,22755.0,24604.5,23630.0,1.677110e+09
86940,2023-03-02 09:00:00,21560.5,23790.0,21683.5,1.677715e+09
97020,2023-03-09 09:00:00,19550.0,26675.0,24344.5,1.678320e+09
